In [13]:
import numpy as np
import sympy as sp
import scipy.special as spc
from scipy.integrate import dblquad

def calculate_dn(n, mu_r, mu_m, R_b, R_r):
    dn_T = R_r**(n+2) + (mu_m * (2*n + 1) * R_b**(n+2) * R_r**(2*n+1)) / \
           ((mu_r - mu_m) * n * R_b**(2*n+1) - (mu_r * n + mu_m * (n + 1)) * R_r**(2*n+1))
    dn_perp = (mu_m - 1) * (n + 1) + (mu_m * (2*n + 1) * (mu_r * n + mu_m * (n + 1)) * R_r**(2*n+1)) / \
              ((mu_r - mu_m) * n * R_b**(2*n+1) - (mu_r * n + mu_m * (n + 1)) * R_r**(2*n+1))
    dn = -dn_T / dn_perp
    return dn

#integration over a spherical surface
def integrate_spherical(n, m, p, M0, alpha0):
    def integrand(phi, theta):
        #associated Legendre polynomials
        P_nm = spc.lpmv(abs(m), n, np.cos(theta))
        #spherical harmonic
        Y_nm = P_nm * np.exp(1j * m * phi)
        #M0r
        M0r = (-1)**(p-1) * M0 * np.cos(phi - alpha0 - (2*np.pi/(2*p)) * (p-1)) * np.sin(theta)
        #real part of the integrand
        return np.real(M0r * Y_nm * np.sin(theta))

    #integrate using scipy's dblquad
    real_part, _ = dblquad(
        integrand,
        0, 2 * np.pi,  #phi limits
        lambda phi: 0,  #theta lower bound as a function of phi
        lambda phi: np.pi,  #theta upper bound as a function of phi
    )

    #If m>0, Im()=0; otherwise, calculate
    imag_part = 0
    if m < 0:
        def integrand_imag(phi, theta):
            P_nm = spc.lpmv(abs(m), n, np.cos(theta))
            Y_nm = P_nm * np.exp(1j * m * phi)
            M0r = (-1)**(p-1) * M0 * np.cos(phi - alpha0 - (2*np.pi/(2*p)) * (p-1)) * np.sin(theta)
            return np.imag(M0r * Y_nm * np.sin(theta))

        imag_part, _ = dblquad(
            integrand_imag,
            0, 2 * np.pi,  #phi limits
            lambda phi: 0,  #theta lbound as a func phi
            lambda phi: np.pi,  #theta ubound as a func phi
        )

    #complex result
    return real_part + 1j * imag_part


#Helper function to compute the spherical harmonics symbolically
def Y_nm_sym(n, m, theta, phi):
    P_nm = sp.assoc_legendre(n, m, sp.cos(theta))
    normalization = sp.sqrt((2*n + 1)/(4*sp.pi) * sp.factorial(n - m)/sp.factorial(n + m))
    return normalization * P_nm * sp.exp(sp.I * m * phi)

In [11]:
import sympy as sp
import numpy as np
import scipy.special as spc
from scipy.integrate import dblquad

#symbolic variables
theta, phi, r = sp.symbols('theta phi r', real=True)
mu0 = sp.symbols('mu0', real=True)

#constants for test
n_value = 4
p_value = 8
M0_value = 1  #magnitude of the residual magnetization vector
alpha0_value = 0
mu_r = 1.05  #relative permeability rotor
mu_m = 1.0   #relative permeability medium (air gap)
R_b = 0.1    #boundary rad in m
R_r = 0.095  #rotor rad in m

#d_n for a given n
d_n = calculate_dn(n_value, mu_r, mu_m, R_b, R_r)

#integration over a spherical surface
C_4_4_real = integrate_spherical(n_value, 4, p_value, M0_value, alpha0_value)
C_4_4_imag = integrate_spherical(n_value, -4, p_value, M0_value, alpha0_value)
C_4_minus_4_real = integrate_spherical(n_value, -4, p_value, M0_value, alpha0_value)
C_4_minus_4_imag = integrate_spherical(n_value, 4, p_value, M0_value, alpha0_value)

#real and imaginary parts to form the complex coefficients
C_4_4 = sp.re(C_4_4_real) + sp.I * sp.re(C_4_4_imag)
C_4_minus_4 = sp.re(C_4_minus_4_real) + sp.I * sp.re(C_4_minus_4_imag)

#a, b, c from C_4_4 and C_4_minus_4
a = sp.re((C_4_4 + C_4_minus_4) / (2 * M0_value))
b = sp.im((C_4_4 - C_4_minus_4) / (2 * M0_value))
c = sp.Abs(C_4_4) / (M0_value * sp.sqrt(a**2 + b**2))

#scalar magnetic potential Phi_i using a, b, c
Phi_i = (C_4_4 * d_n * Y_nm_sym(4, 4, theta, phi) +
         C_4_minus_4 * d_n * Y_nm_sym(4, -4, theta, phi)) * r**(-n_value - 1)

#B_Ir
B_Ir = -mu0 * sp.diff(Phi_i, r)

#simplify
Phi_i_simplified = sp.simplify(Phi_i)
B_Ir_simplified = sp.simplify(B_Ir)


print(f"Phi_i: {Phi_i_simplified}")
print(f"B_Ir: {B_Ir_simplified}")

#numerical evaluation at r=R_b, theta=pi/4, phi=pi/2 DEBBUG
Phi_i_num = Phi_i_simplified.subs({r: R_b, theta: sp.pi/4, phi: sp.pi/2, mu0: 4 * sp.pi * 1e-7})
B_Ir_num = B_Ir_simplified.subs({r: R_b, theta: sp.pi/4, phi: sp.pi/2, mu0: 4 * sp.pi * 1e-7})
print(f"Numerical Phi_i at R_b, pi/4, pi/2: {Phi_i_num.evalf()}")
print(f"Numerical B_Ir at R_b, pi/4, pi/2: {B_Ir_num.evalf()}")

Phi_i: 5.88491444436942e-8*sqrt(70)*(-1 - I)*sin(theta)**4*cos(4*phi)/(sqrt(pi)*r**5)
B_Ir: -2.94245722218471e-7*sqrt(70)*mu0*(1 + I)*sin(theta)**4*cos(4*phi)/(sqrt(pi)*r**6)
Numerical Phi_i at R_b, pi/4, pi/2: -0.00694471209033378 - 0.00694471209033378*I
Numerical B_Ir at R_b, pi/4, pi/2: -4.36349129685776e-7 - 4.36349129685776e-7*I
